In [4]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)

True

In [13]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [6]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model='qwen/qwen3-32b'
)

In [8]:
response = llm.invoke("Hi! Who are you ?")

In [19]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
llm = ChatOpenAI(
    model='gpt-4o-mini'
)
llm.invoke("Hi! Who are you ?")

AIMessage(content="Hello! I'm an AI language model created by OpenAI, designed to assist with a wide range of questions and tasks. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1oH53IQ3lvfsPEyVjvaMZ9BYrVVE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bfadf8dc-c8b2-45e5-a5aa-f9eeb7b280bf-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [34]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
embedding_model = GoogleGenerativeAIEmbeddings(
    model='gemini-embedding-001'
)
llm = ChatGoogleGenerativeAI(
    model='models/gemini-2.0-flash'
)

In [16]:
response = embedding.embed_query('Hello World')

In [17]:
len(response)

3072

In [20]:
openai_embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)
response = openai_embedding.embed_query('Hello World')

In [22]:
len(response)

3072

In [5]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
file_path = os.path.join(os.getcwd(), "data","budget_speech.pdf")
file_path

'/Users/suriyaa/Desktop/NeuroDoc/notebook/data/budget_speech.pdf'

In [ ]:
loader = PyPDFLoader(file_path)
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
)
docs = text_splitter.split_documents(documents)

In [27]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(
    docs,
    embedding_model
)

In [28]:
retriever = vectorstore.as_retriever()

In [31]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [32]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [33]:
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

In [35]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [36]:
response = rag_chain.invoke('Hey what is there for critical mineral sector in this budget?')

In [37]:
response

'In the budget, there are several proposals for the critical minerals sector:\n\n*   Full BCD exemption proposed for cobalt powder and waste, scrap of lithium-ion battery, Lead, Zinc, and 12 more critical minerals to secure their availability for manufacturing in India.\n*   Continuation of the full BCD exemption on 25 critical minerals that are not domestically available (initially exempted in the July 2024 Budget).\n*   Mining sector reforms, including those for minor minerals, will be encouraged through sharing of best practices and institution of a State Mining Index.\n*   A policy for recovery of critical minerals from tailings will be brought out.'